In [8]:
# %pip install anthropic python-dotenv

from dotenv import load_dotenv
load_dotenv()

from anthropic import Anthropic

client = Anthropic()
model = "claude-3-5-haiku-latest"

def add_user_message(messages, text):
  user_message = {"role": "user", "content": text}
  messages.append(user_message)

def add_assistant_message(messages, text):
  assistant_message = {"role": "assistant", "content": text}
  messages.append(assistant_message)

def add_system_message(messages, text):
  system_message = {"role": "system", "content": text}
  messages.append(system_message)


def chat(messages, stop_sequences=[], system=None, temperature=1.0):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [11]:
import json


def generate_dataset():
  prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""
  messages = []
  add_user_message(messages, prompt)
  add_assistant_message(messages, "```json")
  output = chat(messages, stop_sequences=["```"])
  return json.loads(output)

dataset = generate_dataset()
with open("dataset.json", "w") as f:
  json.dump(dataset, f, indent=2)

